# Generative Adversarial Network (GAN)

Generative Adversarial Networks, ou GANs, são redes neurais utilizadas para implementação de modelos generativos, isto é, modelos que geram dados seguindo o padrão dos dados de entrada. 

Essas redes são compostas por dois sub-modelos principais: o modelo gerador e o modelo discriminante.

O modelo gerador constitui a primeira parte das GANs e é utilizado, como sugere o nome, para a geração de novos exemplos baseados nos dados de entrada. Já a segunda parte, o modelo discriminante, é responsável pela discrmininação entre dados reais e falsos (gerados pela rede)

"*Generative adversarial networks are based on a game theoretic scenario in which the generator network must compete against an adversary. The generator network directly produces samples. Its adversary, the discriminator network, attempts to distinguish between samples drawn from the training data and samples drawn from the generator*." - <a href="https://www.amazon.com/Deep-Learning-Adaptive-Computation-Machine/dp/0262035618/ref=as_li_ss_tl?keywords=deep+learning&qid=1553730173&s=books&sr=1-1&linkCode=sl1&tag=inspiredalgor-20&linkId=e9ef92a6248b927689728a8bf72b7cd1&language=en_US" target="_blank">Deep Learning</a>, 2016, página 699.

Neste notebobok, iremos utilizalá-las para a predição de frame de uma cena do filme O Hobbit: Uma Jornada Inesperada (Peter Jackson, 2012) dado um conjunto de frames anteriores da cena como entrada.

In [9]:
import keras
from keras import layers
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob

### Modelo generativo

In [4]:
latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

# Resize das imagens de entrada para 16x16 com 128 canais/features
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# Camada convolucional
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Camada convolucional para sobreamostragem das imagens de entrada para 32x32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

# Camadas convolucionais adicionais
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Transformando para imagem 32x32 com um canal único
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       104883

### Modelo discriminante

In [5]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

# Camada de dropout
x = layers.Dropout(0.4)(x)

# Camada de classificação
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

# Otimizador do treinamento da rede com taxa de aprendizagem (lr)
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)         0   

### Adversarial network

Aqui, o modelo discriminante é posto em ação, sendo utlizado para discriminar as imagens entre reais e falsas.

In [6]:
# Não treinar pesos do modelo discriminante
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

### Carregamento dos dados e treinamento da rede

Para cada iteração do treino:
* Gerar ruído aleatório - dados aleatórios para inicialização de vetores.
* Gerar imagens com o modelo gerador com esses dados aleatórios.
* Combinar as imagens geradas com as imagens reais.
* Treinar o discriminante com essas imagens combinadas, separando entre reais e falsas
* Gerar novos vetores aleatórios
* Treinar rede GAN utilizando apenas esses últimos vetores aleatórios, com rótulos "reais" em todos em eles.

In [7]:
import os
from keras.preprocessing import image

In [11]:
# Carregando os dados
filepath = glob('drive/MyDrive/Colab Notebooks/train/*.jpg')
x_train = np.array([np.array(Image.open(x)) for x in filepath])

KeyboardInterrupt: ignored

In [1]:
# Normalizando os dados
x_train = x_train.astype('float32') / 255.

NameError: ignored

In [23]:
iterations = 500
batch_size = 20
save_dir = '/'

In [20]:
start = 0
for step in range(iterations):
    # Setando vetores alelatórios
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Predição de imagens falsas
    generated_images = generator.predict(random_latent_vectors)

    # Combinando com imagens reais
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # Rotulando dados entre reais e falsos
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # Adicionando ruído aleatório
    labels += 0.05 * np.random.random(labels.shape)

    # Treinando o otimizador
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # Setando vetores alelatórios
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    misleading_targets = np.zeros((batch_size, 1))

    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # Salvando pesos e imagens geradas a cada 10 iterações
    if step % 10 == 0:
        gan.save_weights('gan.h5')

        print('discriminator loss at step %s: %s' % (step, d_loss))
        print('adversarial loss at step %s: %s' % (step, a_loss))

        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))

        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))

discriminator loss at step 0: 0.6879838705062866
adversarial loss at step 0: 0.748475193977356


FileNotFoundError: ignored

In [22]:
os.getcwd()

'/content'

In [24]:
os.path.join(save_dir, 'generated_frog' + str(step) + '.png')

'/generated_frog0.png'